In [6]:
import cv2
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from deepface import DeepFace

# Define the 13 facial regions
facial_regions = [
    'Forehead', 'Right Eyebrow','Left Eyebrow', 'Right Eye', 'Left Eye',
    'Inner Brows', 'Nasion', 'Right Cheek', 'Left Cheek', 'Right Nasalobial',
    'Left Nasalobial', 'Mouth', 'Chin'
]

# Read the video file
cap = cv2.VideoCapture("wap_video.mp4")
ret, prev_frame = cap.read()
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Initialize lists to store feature vectors and labels
feature_vectors = []
labels = []
# Initialize a dictionary to store the mapping between feature vectors and labels
#feature_label_mapping = {}

# Convert the frame to grayscale
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Create a mask for drawing the optical flow vectors
mask = np.zeros_like(prev_frame)

# Load the trained decision tree model
classifier_tree = DecisionTreeClassifier(criterion='entropy', random_state=0)

while True:
    # Read the next frame
    ret, frame = cap.read()

    if not ret:
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Compute optical flow using the Lucas-Kanade method
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Perform face detection
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Loop over the detected faces
    for (x, y, w, h) in faces:
        # Loop over the facial regions
        for region in facial_regions:
            # Extract the region of interest (ROI)
            roi = frame[y:y+h, x:x+w]

            # Compute the optical flow vectors within the ROI
            roi_feature_vectors = []
            for y_roi in range(y + 130, y + 170, 10):
                for x_roi in range(x + 250, min(x + 350, gray.shape[1] - 1), 10):  # Add check for valid x_roi
                    dx, dy = flow[y_roi, x_roi]
                    roi_feature_vectors.append((dx, dy))

            '''for y_roi in range(y + 130, y + 170, 10):
                for x_roi in range(x + 250, x + 350, 10):
                    dx, dy = flow[y_roi, x_roi]
                    roi_feature_vectors.append((dx, dy))'''

            # Detect expressions using DeepFace
            result = DeepFace.analyze(roi, actions=['emotion'], enforce_detection=False)

            # Get the emotion probabilities
            #emotion_probs = result[0]['emotion']['probabilities']
            dominant_emotion = result[0]['dominant_emotion']
            #emotion_l = result['emotion'][0]

            # Append the feature vectors and corresponding labels
            feature_vectors.append(roi_feature_vectors)
            #labels.append(emotion_probs)
            labels.append(dominant_emotion)
            #feature_label_mapping[tuple(roi_feature_vectors)] = dominant_emotion
            # Append the feature vectors and corresponding labels
            #feature_vectors.append(roi_feature_vectors)

    # Update the previous frame and previous gray image
    prev_frame = frame
    prev_gray = gray

# Convert feature vectors and labels to numpy arrays
# Find the maximum length of feature vectors
max_length = max(len(vector) for vector in feature_vectors)

# Pad the shorter feature vectors with zeros
padded_feature_vectors = []
for vector in feature_vectors:
    padding = max_length - len(vector)
    padded_vector = vector + [(0, 0)] * padding
    padded_feature_vectors.append(padded_vector)

# Convert the padded feature vectors and labels to NumPy arrays
feature_vectors = np.array(padded_feature_vectors)
#feature_vectors = np.array(feature_vectors)
labels = np.array(labels)
print("Labels:",labels.shape)
print("feature_vectors:",feature_vectors.shape)
#print(feature_vectors)

# Reshape the feature vectors for training
#feature_vectors = feature_vectors.reshape(len(feature_vectors), -1)
feature_vectors = feature_vectors.reshape(2288, 40)
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(feature_vectors, labels, test_size=0.2, random_state=42)
print("Labels:",labels.shape)
print("feature_vectors:",feature_vectors.shape)
print("x:",X_train.shape)
print("y:",y_train.shape)
# Train the decision tree model
classifier_tree.fit(X_train, y_train)
distress_prediction = classifier_tree.predict(X_test)
print("Distress Prediction:",distress_prediction)
c=0
d=0
for (x, y, w, h) in faces:
    # Predict the distress using the decision tree model
        
                # Update the counters based on the predicted emotions
        if any(dominant_emotion in distress_prediction for dominant_emotion in ['Anger', 'Disgust', 'Fear', 'Surprised']):
                c += 1

        if any(dominant_emotion in distress_prediction for dominant_emotion in ['Sad','Happy', 'Neutral']):
                d += 1
        

if c > d:
    print(c)
    print("Distress Detected")
else:
    print(d)
    print("Neutral Face")
# Evaluate the model on the testing set
#accuracy = classifier_tree.score(X_test, y_test)
#print("Accuracy:", accuracy)

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()


Action: emotion: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 13.81it/s]


Labels: (2288,)
feature_vectors: (2288, 20, 2)
Labels: (2288,)
feature_vectors: (2288, 40)
x: (1830, 40)
y: (1830,)
Distress Prediction: ['neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'fear'
 'neutral' 'fear' 'neutral' 'happy' 'sad' 'fear' 'fear' 'neutral' 'fear'
 'neutral' 'neutral' 'neutral' 'neutral' 'fear' 'neutral' 'neutral'
 'neutral' 'neutral' 'neutral' 'neutral' 'fear' 'neutral' 'fear' 'fear'
 'fear' 'neutral' 'neutral' 'neutral' 'neutral' 'fear' 'fear' 'neutral'
 'neutral' 'neutral' 'neutral' 'fear' 'neutral' 'fear' 'neutral' 'neutral'
 'fear' 'fear' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral'
 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'fear' 'neutral' 'fear'
 'neutral' 'neutral' 'neutral' 'fear' 'neutral' 'neutral' 'neutral'
 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral'
 'fear' 'fear' 'fear' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral'
 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral'
 'n

In [8]:
import cv2
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from deepface import DeepFace

# Define the 13 facial regions
facial_regions = [
    'Forehead', 'Right Eyebrow','Left Eyebrow', 'Right Eye', 'Left Eye',
    'Inner Brows', 'Nasion', 'Right Cheek', 'Left Cheek', 'Right Nasalobial',
    'Left Nasalobial', 'Mouth', 'Chin'
]

# Read the video file
cap = cv2.VideoCapture("distress.mp4")
ret, prev_frame = cap.read()
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Initialize lists to store feature vectors and labels
feature_vectors = []
labels = []
# Initialize a dictionary to store the mapping between feature vectors and labels
#feature_label_mapping = {}

# Convert the frame to grayscale
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Create a mask for drawing the optical flow vectors
mask = np.zeros_like(prev_frame)

# Load the trained decision tree model
classifier_tree = DecisionTreeClassifier(criterion='entropy', random_state=0)

while True:
    # Read the next frame
    ret, frame = cap.read()

    if not ret:
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Compute optical flow using the Lucas-Kanade method
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Perform face detection
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Loop over the detected faces
    for (x, y, w, h) in faces:
        # Loop over the facial regions
        for region in facial_regions:
            # Extract the region of interest (ROI)
            roi = frame[y:y+h, x:x+w]

            # Compute the optical flow vectors within the ROI
            roi_feature_vectors = []
            for y_roi in range(y + 130, y + 170, 10):
                for x_roi in range(x + 250, min(x + 350, gray.shape[1] - 1), 10):  # Add check for valid x_roi
                    dx, dy = flow[y_roi, x_roi]
                    roi_feature_vectors.append((dx, dy))

            '''for y_roi in range(y + 130, y + 170, 10):
                for x_roi in range(x + 250, x + 350, 10):
                    dx, dy = flow[y_roi, x_roi]
                    roi_feature_vectors.append((dx, dy))'''

            # Detect expressions using DeepFace
            result = DeepFace.analyze(roi, actions=['emotion'], enforce_detection=False)

            # Get the emotion probabilities
            #emotion_probs = result[0]['emotion']['probabilities']
            dominant_emotion = result[0]['dominant_emotion']
            #emotion_l = result['emotion'][0]

            # Append the feature vectors and corresponding labels
            feature_vectors.append(roi_feature_vectors)
            #labels.append(emotion_probs)
            labels.append(dominant_emotion)
            #feature_label_mapping[tuple(roi_feature_vectors)] = dominant_emotion
            # Append the feature vectors and corresponding labels
            #feature_vectors.append(roi_feature_vectors)

    # Update the previous frame and previous gray image
    prev_frame = frame
    prev_gray = gray

# Convert feature vectors and labels to numpy arrays
# Find the maximum length of feature vectors
max_length = max(len(vector) for vector in feature_vectors)

# Pad the shorter feature vectors with zeros
padded_feature_vectors = []
for vector in feature_vectors:
    padding = max_length - len(vector)
    padded_vector = vector + [(0, 0)] * padding
    padded_feature_vectors.append(padded_vector)

# Convert the padded feature vectors and labels to NumPy arrays
feature_vectors = np.array(padded_feature_vectors)
#feature_vectors = np.array(feature_vectors)
labels = np.array(labels)
print("Labels:",labels.shape)
print("feature_vectors:",feature_vectors.shape)
#print(feature_vectors)

# Reshape the feature vectors for training
#feature_vectors = feature_vectors.reshape(len(feature_vectors), -1)
feature_vectors = feature_vectors.reshape(130, 80)
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(feature_vectors, labels, test_size=0.2, random_state=42)
print("Labels:",labels.shape)
print("feature_vectors:",feature_vectors.shape)
print("x:",X_train.shape)
print("y:",y_train.shape)
# Train the decision tree model
classifier_tree.fit(X_train, y_train)
distress_prediction = classifier_tree.predict(X_test)
print("Distress Prediction:",distress_prediction)
c=0
d=0
for (x, y, w, h) in faces:
    # Predict the distress using the decision tree model
        
                # Update the counters based on the predicted emotions
        if any(dominant_emotion in distress_prediction for dominant_emotion in ['Anger', 'Disgust', 'Fear', 'Surprised']):
                c += 1

        if any(dominant_emotion in distress_prediction for dominant_emotion in ['Sad','Happy', 'Neutral']):
                d += 1
        

if c > d:
    print(c)
    print("Distress Detected")
else:
    print(d)
    print("Neutral Face")
# Evaluate the model on the testing set
#accuracy = classifier_tree.score(X_test, y_test)
#print("Accuracy:", accuracy)

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()


Action: emotion: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  8.29it/s]


Labels: (130,)
feature_vectors: (130, 40, 2)
Labels: (130,)
feature_vectors: (130, 80)
x: (104, 80)
y: (104,)
Distress Prediction: ['happy' 'happy' 'sad' 'sad' 'happy' 'happy' 'happy' 'happy' 'happy' 'sad'
 'happy' 'sad' 'happy' 'fear' 'happy' 'happy' 'sad' 'happy' 'happy'
 'happy' 'sad' 'fear' 'happy' 'fear' 'happy' 'happy']
0
Neutral Face
